In [1]:
import os
import torch
import torchvision
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
# from torch.utils.tensorboard import summary
from torchvision.utils import save_image
from tqdm import tqdm

import matplotlib.pyplot as plt
import numpy as np

from model import ByteNetLMTime
from dataset import ProteinSequenceDataset, Collater
from losses import D3PMLVBLoss
from utils import Tokenizer

In [2]:
gpu = torch.cuda.is_available()
device = torch.device("cuda:0" if gpu else "cpu")
print("device:", device)

device: cpu


In [3]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [4]:
# seed for reproducability
torch.manual_seed(50)

# training parameters
diffusion_timesteps = 500
num_epochs = 10
batch_size = 128
lr = 2e-4
# model = UNet(image_channels=1, model_output='logistic_pars').to(device)
# optimizer = optim.Adam(model.parameters(), lr=lr)

In [5]:
data_train = ProteinSequenceDataset(train=True)
data_test = ProteinSequenceDataset(train=False)

tokenizer = Tokenizer()
Q_prod, Q_t = tokenizer.q_uniform_schedule(num_steps=diffusion_timesteps)
collater = Collater(tokenizer=tokenizer, num_steps=diffusion_timesteps, Q=Q_t, Q_bar=Q_prod)

In [6]:
train_loader = DataLoader(dataset=data_train, batch_size=batch_size, collate_fn=collater)

In [7]:
data_iter = iter(train_loader)
batch = next(data_iter)

KeyError: '------TLMTLQAKYNVLFGIAAVDTRGGRTF-EWNADDRFAMCSTFKVYAVAAILRLAERRELDLNDKVPITRQDIVMNSPVSYDHIGELMSYYELSRAALVRSDNTAANLLLRRLGGPAAVTAFARSINDSQTRLDRWEPELNEARAGDSRDTSTASAFAAGYRKLILGDGLSTEYRRMLTSWMRSSVTSRQRLRAGLPVGWVAADKTGSGAYGTANDAGVVWGPTGTSLVLVIFSRSTTGYAAGADAAVADAAAAVL---'